In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
from numpy import nan

### Useful variables for data gen & plotting

In [ ]:
# for plotly to make font sizes bigger
layout_update_dict = dict(
    showlegend=False,
    font={'size': 20},
    title={'font': {'size': 24}}
)

# for selecting the relevant columns for plotting
cols_dict = {
    'filing_yr': 'max',
    'grantspossession': 'bool_or',
    'judgement_for_tenant': 'bool_or',
    'monthlyrentamount': 'max',
    'totaljudgmentamount': 'max',
    'casestatus': 'max',
    'settled': 'bool_or',
    'withdrawn': 'bool_or',
    'dismissed': 'bool_or',
    'claimamount': 'max',
}

def make_select_str(d):
    return ', '.join(['{agg_fcn}({col_name}) as {col_name}'.format(agg_fcn=agg_fcn, col_name=col_name) for col_name, agg_fcn in d.items()])

col_select_str = make_select_str(cols_dict)

# connection to acdhs-housing db
acdhs_engine = create_engine("postgresql:///acdhs-housing")

In [ ]:
query = """select {}
from clean.eviction e
group by matter_id 
""".format(col_select_str)

case_level_data = pd.read_sql(query, acdhs_engine)

In [ ]:
case_level_data.summary()

### How many evictions are there?

In [ ]:
len(case_level_data)

### How many per year? How many are won by tenants (red) vs landlords (purple)?

In [ ]:
fig = px.histogram(case_level_data, x='filing_yr', title='Number of evictions filed per year', color='judgement_for_tenant', text_auto=True)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Number of evictions', bargap=0.2, yaxis_range=[0, 16000], width=1500, height=500)
fig.show()

### Of non-active cases, how many are pay-and-stay (red) vs possession (purple)?
We look at non-active since possession is decided when the judge makes a decision

In [ ]:
actually_tried_cases = case_level_data[(case_level_data.casestatus.isin(['Closed', 'Inactive'])) & (case_level_data.withdrawn == 0) & (case_level_data.dismissed == 0)]
actually_tried_cases['grantspossession'] = actually_tried_cases.grantspossession.replace(nan, 'Unknown')
fig = px.histogram(actually_tried_cases, x='filing_yr', title='Type of evictions filed per year', color='grantspossession', text_auto=True)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Number of evictions', bargap=0.2, yaxis_range=[0, 16000], width=1500, height=500)
fig.show()

### How much are landlords claiming they're owed?

In [ ]:
case_level_data['grantspossession'] = case_level_data.grantspossession.replace(nan, 'Unknown')
fig = px.histogram(case_level_data, x='claimamount', title='$ amount claimed per eviction', color='grantspossession')
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Number of evictions')
fig.show()

### How much is the monthly rent for these places?

In [ ]:
fig = px.histogram(case_level_data, x='monthlyrentamount', title='monthly rent amount per eviction', color='judgement_for_tenant', log_y=True, nbins=750)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Monthly rent', yaxis_title='Number of evictions (log scale)')
fig.show()

### The pandemic obviously affected evictions. Let's look at monthly data from 2020 onwards.

In [ ]:
count_booleans_agg = lambda col_name: 'count(case when {} then 1 end)'.format(col_name)

cols_dict = { col_name: count_booleans_agg(col_name) for col_name in ('grantspossession', 'judgement_for_tenant', 'settled', 'withdrawn', 'dismissed' ) }

def make_select_str(d):
    return ', '.join(['{} as {}'.format(agg_fcn, col_name) for col_name, agg_fcn in d.items()])

col_select_str = make_select_str(cols_dict)

query = """ select extract(year from filingdt) as filing_yr, extract(month from filingdt) as filing_month, count(*), {}
from clean.eviction e
where extract(year from filingdt) >= 2020
group by extract(year from filingdt), extract(month from filingdt)""".format(col_select_str)

pandemic_data = pd.read_sql(query, acdhs_engine, parse_dates=['last_updated_ts'])
pandemic_data['date'] = pd.to_datetime(pandemic_data.apply(lambda row: '{}-{:02d}-01'.format(int(row.filing_yr), int(row.filing_month)), axis=1))
pandemic_data['pay-and-stay'] = pandemic_data['count'] - pandemic_data.grantspossession

In [ ]:
fig = px.bar(pandemic_data, x='date', y=['grantspossession', 'pay-and-stay'], title='Type of evictions filed during the pandemic', text_auto=False)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Number of evictions')
fig.update_xaxes(tickangle=45)
fig.show()

### We know there are all sorts of fees associated with evictions. What are they and how often do they come up?

In [ ]:
query = """select civiljudgmentcomponenttype, count(*)
from clean.eviction e
where casestatus = 'Closed'
group by civiljudgmentcomponenttype 
"""

payment_types_count = pd.read_sql(query, acdhs_engine)

In [ ]:
fig = px.bar(payment_types_count, x='civiljudgmentcomponenttype', y='count', title='Frequency of different payment types', text_auto=False)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Payment types', yaxis_title='Count', height=800)
fig.update_xaxes(tickangle=30)
fig.show()

In [ ]:
### If we were able to pay overdue rent before evictions are taken to court, how much could we possibly save?

In [ ]:
query = """select extract(year from filingdt) as filing_yr, sum(totaljudgmentamount) as totaljudgmentamount, sum(component_amount) as component_amount, sum(totaljudgmentamount) - sum(component_amount) as total_judgment_minus_overdue_rent
from clean.eviction
where civiljudgmentcomponenttype = 'Rent in Arrears' and casestatus='Closed'
group by extract(year from filingdt)"""

money_by_year_data = pd.read_sql(query, acdhs_engine)

In [ ]:
fig = px.bar(money_by_year_data, x='filing_yr', y=['component_amount', 'totaljudgmentamount'], barmode='group', title='Rent in Arrears and Total Judgement Cost by Year', text_auto=False)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Amount (USD)')
fig.show()

In [ ]:
fig = px.bar(money_by_year_data, x='filing_yr', y='total_judgment_minus_overdue_rent', barmode='group', title='Total money judged - rent in arrears per year', text_auto=False)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Amount judged - claimed (USD)')
fig.show()

### What do we have in eviction landlord data?
i.e. how many landlords are there?

In [ ]:
query = 'select count(distinct participant_id) from clean.eviction_landlords'

print(pd.read_sql(query, acdhs_engine)['count'].values[0], 'unique landlords for', len(case_level_data), 'eviction cases' )

### Are the same landlords filing evictions over again?

In [ ]:
query = """select e1.participant_id, count(*), max(unique_displaynm) as unique_displaynm
from (select e.matter_id, max(e.filingdt) as filingdt, max(el.participant_id) as participant_id, max(el.unique_displaynm) as unique_displaynm
	from clean.eviction e
	left join clean.eviction_landlords el on e.matter_id = el.matter_id
	group by e.matter_id) as e1
where participant_id is not null
group by e1.participant_id"""

landlords_frequency = pd.read_sql(query, acdhs_engine)

In [ ]:
fig = px.histogram(landlords_frequency, x='count', title='frequency of evictions filed by landlords', log_y=True)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Monthly rent', yaxis_title='Number of evictions (log scale)')
fig.show()

### So who are the top landlords evicting so many people?

Are they spread throughout the years, or are there spikes in evictions from these landlords?

In [ ]:
for _, row in landlords_frequency.sort_values(['count'], ascending=False).head(10).iterrows():
    print(row.unique_displaynm, 'has evicted', row['count'], 'people ( participant id', row.participant_id, ')')

In [ ]:
# choosing one participant
participant_id = 624600919

query = """select extract(year from filing_yr) as filing_yr, count(*)
from clean.eviction_landlords el
left join clean.eviction e on el.matter_id=e.matter_id 
where participant_id = {} 
group by extract(year from filing_yr)
""".format(participant_id)

landlord_by_yr = pd.read_sql(query, acdhs_engine)


In [ ]:
fig = px.bar(landlord_by_yr, x='filing_yr', y='count', title='Type of evictions filed per year', text_auto=True)
fig.update_layout(layout_update_dict)
fig.update_layout(xaxis_title='Filing year', yaxis_title='Number of evictions', bargap=0.2, width=1500, height=500)
fig.show()